In [3]:
import os
from langchain.vectorstores import Chroma
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import AnalyzeDocumentChain, RetrievalQA
from langchain.chains.summarize import load_summarize_chain

embedding = OpenAIEmbeddings()
llm = OpenAI(temperature=0)

persist_directory = 'db_langchain'
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)
retriever = vectordb.as_retriever(search_type="mmr")

# summary_chain = load_summarize_chain(OpenAI(temperature=0), chain_type="map_reduce")
# summarize_document_chain = AnalyzeDocumentChain(combine_docs_chain=summary_chain)

qa = RetrievalQA.from_chain_type(llm=OpenAI(temperature=0), chain_type="stuff", retriever=retriever, verbose=True,)

Using embedded DuckDB with persistence: data will be stored in: db_langchain


In [10]:
docs  = vectordb.similarity_search("What methods are on the Agent Class?")

In [22]:
print(docs[0].page_content)

.


Agent Types


For documentation on how to create a custom agent, see the below.


Custom Agent
Custom LLM Agent
Custom LLM Agent (with a ChatModel)
Custom MRKL Agent
Custom MultiAction Agent
Custom Agent with Tool Retrieval


We also have documentation for an in-depth dive into each agent type.


Conversation Agent (for Chat Models)
Conversation Agent
MRKL
MRKL Chat
ReAct
Self Ask With Search
Structured Tool Chat Agent










previous
Zapier Natural Language Actions API




next
Agent Types












By Harrison Chase




    
      Â© Copyright 2023, Harrison Chase.
      




  Last updated on May 25, 2023.
  


















\n\nAgents\nGetting Started\nKey Concepts\nHow-To Guides\nAgents and Vectorstores\nAsync API for Agent\nConversation Agent (for Chat Models)\nChatGPT Plugins\nCustom Agent\nDefining Custom Tools\nHuman as a tool\nIntermediate Steps\nLoading from LangChainHub\nMax Iterations\nMulti Input Tools\nSearch Tools\nSerialization\nAdding SharedMemory to an Agent and its Tools\nCSV Agent\nJSON Agent\nOpenAPI Agent\nPandas Dataframe Agent\nPython Agent\nSQL Database Agent\nVectorstore Agent\nMRKL\nMRKL Chat\nReAct\nSelf Ask With Search\n\n\nReference\n\n\nMemory\nGetting Started\nKey Concepts\nHow-To Guides\nConversationBufferMemory\nConversationBufferWindowMemory\nEntity Memory\nConversation Knowledge Graph Memory\nConversationSummaryMemory\nConversationSummaryBufferMemory\nConversationTokenBufferMemory\nAdding Memory To an LLMChain\nAdding Memory to a Multi-Input Chain\nAdding Memory to an Agent\nChatGPT Clone\nConversation Agent\nConversational Memory Customization\nCustom Memory\nMultiple Mem

In [23]:
import ast
import astor

def extract_definitions(source_code):
    module = ast.parse(source_code)
    classes = []
    functions = []

    for node in module.body:
        if isinstance(node, ast.ClassDef):
            classes.append({
                'name': node.name,
                'source': ast.unparse(node)
            })
        elif isinstance(node, ast.FunctionDef):
            functions.append({
                'name': node.name,
                'source': ast.unparse(node)
            })

    return classes, functions

source_code = """
# Your source code here
"""

classes, functions = extract_definitions(source_code)

for cls in classes:
    print(f"Class name: {cls['name']}")
    print(f"Source code:\n{cls['source']}\n")

for func in functions:
    print(f"Function name: {func['name']}")
    print(f"Source code:\n{func['source']}\n")


In [26]:
with open("BabyAGI.py", "r") as f:
    text = f.read()

classes, functions = extract_definitions(text)
for cls in classes:
    print(f"Class name: {cls['name']}")
    print(f"Source code:\n{cls['source']}\n")

for func in functions:
    print(f"Function name: {func['name']}")
    print(f"Source code:\n{func['source']}\n")

OSError: [Errno 63] File name too long: 'from langchain import OpenAI, GoogleSearchAPIWrapper, LLMChain\nfrom langchain.agents import ZeroShotAgent, Tool, AgentExecutor\nimport faiss\nimport json\nimport os\nfrom collections import deque\nfrom typing import Dict, List, Optional, Any\n\nfrom langchain import LLMChain, OpenAI, PromptTemplate\nfrom langchain.embeddings import OpenAIEmbeddings\nfrom langchain.llms import BaseLLM\nfrom langchain.vectorstores.base import VectorStore\nfrom pydantic import BaseModel, Field\nfrom langchain.chains.base import Chain\n\nfrom langchain.vectorstores import FAISS\nfrom langchain.docstore import InMemoryDocstore\n\nimport firebase_admin\nfrom firebase_admin import credentials\nfrom firebase_admin import firestore\n\n# Use a service account\n# cred = credentials.Certificate(\'cfa-creds.json\')\n# firebase_admin.initialize_app(cred)\n\n# db = firestore.client()\n\n# Define your embedding model\nembeddings_model = OpenAIEmbeddings()\n\n# Initialize the v